In [1]:
import tensorflow as tf
from sqlalchemy import create_engine
import sqlalchemy as db
import pandas as pd
import numpy as np
from tensorflow.python.client import device_lib
from keras.models import load_model
import gc
import onnx
import onnxruntime as rt

from mclLib.featurenames import *
from mclLib.cleaner import *
from mclLib.ftploader import *
from mclLib.modelpostfix import *
from mclLib.tester import *
from mclLib.server_info import *

In [2]:
print(device_lib.list_local_devices())

engine = create_engine(mysql_server_uri)
conn = engine.connect()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12920598705928583544
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 510866228
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17889900938811050350
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 750 Ti, pci bus id: 0000:01:00.0, compute capability: 5.0"
xla_global_id: 416903419
]


In [3]:
''' get db data '''
br_full_data = pd.read_sql_table('buyreports', conn)

In [4]:
br = br_full_data

In [5]:
''' db data filter '''
get_filter = ( br_full_data['isAllBuyed'] == 1) & ( br_full_data['isAllSelled'] == 1) 
br = br_full_data[get_filter]

In [7]:
''' get features name'''
feature_names =  f_name_102
feature_size = len(feature_names)

''' set X data '''
X = br[feature_names].to_numpy(dtype=np.float64)

''' set y data '''
y_condition = (br['fMaxPowerAfterBuyWhile10'] < 0.025)
y = np.where(y_condition, 1, 0)

y_test =y
y_predict = []


In [8]:
# 테스트할 모델

scale_no_model_names = [
  # 'RF_pro_01_n150_md8', 'RF_pro_01_n200_md10', 'RF_pro_01_n100_md12', 'RF_pro_01_n100_md5', 'RF_pro_01_n100_md6', 
  # 'RF_pro_012_n100_md6', 'RF_pro_012_n120_md10',
  # 'RF_pro_014_n100_md5', 'RF_pro_014_n200_md10',
  # 'RF_pro_016_n100_md5', 'RF_pro_016_n100_md6','RF_pro_016_n150_md8',
  # 'RF_pro_018_n100_md7', 'RF_pro_018_n150_md6',
  # 'RF_pro_02_n100_md5', 'RF_pro_02_n100_md8','RF_pro_02_n150_md9',


  'rf_max30_d4_v2', 'rf_maxM_d2_v2', 'rf_maxM_d3_v1', 'rf_maxM_d3_v2', 'rf_maxM_d45_v2',
  'RF_pow10_02_n100_md10', 'RF_pow10_003_n150_md4', 'RF_pow10_003_n200_md7', 'RF_pow10_004_n300_md6',
  'RF_pow10_004_n300_md6_randover04', 'RF_pow10_004_n300_md6_smote04', 'RF_pow10_04_n100_md7', 
  'RF_pow10_04_n100_md10', 'RF_pow10_025_n100_md10', 'RF_pow10_026_n120_md10', 'RF_pow30_02_n120_md5_v2',
  'RF_pow30_02_n120_md7_v2', 'RF_pow30_02_n120_md10_v2', 'RF_pow30_003_n100_md5', 'RF_pow30_03_n120_md5_v2',
  'RF_pow30_03_n120_md7_v2', 'RF_pow30_03_n120_md8_v2', 'RF_pow30_03_n120_md10_v2', 'RF_pow30_04_n60_md7',
  'RF_pow30_04_n100_md10', 'RF_pow30_04_n150_md7', 'RF_pow30_025_n120_md5_v2', 'RF_pow30_025_n120_md7_v2',
  'RF_pow30_025_n120_md8_v2', 'RF_pow30_025_n120_md10_v2', 'RF_pow30_0035_n100_md5', 'RF_pow30_035_n120_md5_v2',
  'RF_pow30_035_n120_md7_v2', 'RF_pow30_035_n120_md8_v2', 'RF_pow30_035_n120_md10_v2',


  # 'RF_pow_05_n100_md5', 'RF_pow_05_n120_md9', 'RF_pow_05_n150_md9', 'RF_pow_05_n50_md10', 'RF_pow_05_n50_md8',
  # 'RF_pow_055_n100_md6', 'RF_pow_055_n120_md12', 'RF_pow_055_n200_md12', 'RF_pow_055_n200_md7',
  #  'RF_pow_06_n250_md10'
]


In [9]:
ftp= FtpLoader(ftp_ip, ftp_port, ftp_id, ftp_pw)

In [10]:
for idx, mn in enumerate(scale_no_model_names):
    ftp.download(mn+onnx_, onnx_path, just_onnx)
    
    x_datas = X

    # onnx_model = onnx.load(onnx_path + mn + onnx_)
    sess = rt.InferenceSession(onnx_path + mn + onnx_)
    res = sess.run(['output'],{'input': x_datas})
    y_predict.append(res[0])
    
    del sess
    gc.collect()

In [11]:
#################################
############## test #############

# suc_crit = 0.8 # 1이라 판정할 값의 기준
# fl_crit = 0.5  # 0이라 판정할 값의 기준

suc_ratio = 0.65 # 모델들에서 1이라 종합할 비율
fail_ratio = .65 # 모델들에서 0이라 종합할 비율

testClassification(y_test, y_predict, suc_pass_ratio=suc_ratio, fail_pass_ratio=fail_ratio)


분류평가를 시작합니다...
len of models :  35
suc_line : 23 (0.65)
fail_line : 23 (0.65)
crit value :  0.5

총량 :  14265
0 :  4356 , 비율 :  30.536277602523658 (%)
1 :  9909 , 비율 :  69.46372239747633 (%)

============ predict 0 =============
총 횟수 :  205
실제 0 :  154
실제 1 :  51
정답비율 :  75.1219512195122 (%)

============ predict 1 =============
총 횟수 :  13602
실제 1 :  9679
실제 0 :  3923
정답비율 :  71.15865313924424 (%)



([1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,


In [12]:
removeTestData(onnx_list=scale_no_model_names)